In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from pgmpy.estimators import HillClimbSearch
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BicScore
from pgmpy.base import DAG
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn import metrics
from pgmpy.base import DAG
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
from rpy2.robjects import globalenv
from rpy2.robjects.packages import importr
from pgmpy.estimators import K2Score
from rpy2.robjects import conversion, default_converter, pandas2ri
from rpy2.robjects.conversion import localconverter
from pgmpy.models import BayesianNetwork
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import rpy2.robjects as ro
from pgmpy.estimators import IVEstimator
from pgmpy.estimators import ExpectationMaximization as EM
from pgmpy.estimators import BayesianEstimator as BE
from pgmpy.estimators import BDeuScore
from pgmpy.estimators import BicScore
from sklearn.metrics import mutual_info_score
from sklearn.metrics import roc_auc_score
pandas2ri.activate()
from sklearn.metrics import f1_score

1. 更改融合算法
2. 函数跳出出口更改

In [2]:
def to_matrix(edges_dag,columns):
    #将图转换为矩阵
    edges_matrix = pd.DataFrame(np.zeros((len(columns),len(columns))),index = columns,columns=columns).astype(int)
    for edges in edges_dag:
        edges_matrix.loc[edges[0],edges[1]]+=1
    return edges_matrix

def to_dag(edges_matrix):
    edges_list = []
    for index in edges_matrix.index:
        for col in edges_matrix.columns:
            if edges_matrix.loc[index,col]!=0:
                edges_list.append((index,col))
            
    #将矩阵转换为图
    return edges_list  
def print_dag(dag,all_node=False):
    if not all_node:
        if dag.has_node("label"):
            for edges in dag.get_ancestral_graph("label").edges:
                print(edges[0],"->",edges[1],";")
        else:
            print("label have not ancestor")
            for edges in dag.edges:
                print(edges[0],"->",edges[1],";")
    else:
         for edges in dag.edges:
                print(edges[0],"->",edges[1],";")
    return 
            
def model_auc(prob,y_true):
    y_prob = []
    num = 0
    for i in range(len(prob)):
        y_prob.append(prob.iloc[i,y_true[i]])
    if y_predic[i]==y_true[i]:
        num+=1
    fpr, tpr, thresholds = roc_curve(y_true,y_predic)
    
    return auc(fpr, tpr)
def to_strength_list(edges_strength_martix):
    str_list = []
    for col in edges_strength_martix.columns:
        for idx in edges_strength_matrix.columns:
            #不等于才加
            if edges_strength_matrix[col][idx]!=0:
                str_list.append([idx,col,edges_strength_matrix[col][idx]])
    return str_list

#根据互信息的分数计算黑名单
def caculate_hos_bl(data,bl):    
    mi_bl = []
    for col in data.columns:
        for index in data.columns:
            mi_score = mutual_info_score(data.loc[:,index],data.loc[:,col])
            if col==index:
                continue
            ##互信息分数，如果两个特征的互信息分数小雨0.02的话就将它加入黑名单
            if mi_score<0.005:
                mi_bl.append([index,col])
    h_bl=pd.DataFrame(mi_bl,columns=["from","to"])
    h_bl=pd.concat([h_bl,bl])
   
    return h_bl

def net_score(ed,data,hos_id,scr):
    

        
  
    ##转换为R字符串
    r_dag = py_to_rdag(ed)
    ##预测
    r_y = ro.r['predict_label'](r_dag,data,hos_id)-1
    for i in range(len(r_y)):
        if r_y[i]<0:
            r_y[i]=0
        elif r_y[i]>1:
            r_y[i]=1

    split_size = int(len(r_y)*0.5) 
    
    y_pred = r_y[:split_size]
    
    y_true = r_y[split_size:]
    
    acc = accuracy_score(y_true,y_pred)
    recall = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)
    
    dag_scr = ro.r["dag_score"](r_dag,DB,scr,hos_id)[0]
    
    return acc,recall,auc,f1,dag_scr
def py_to_rdag(dag):
    rdag_str = ""
    for node in dag.nodes():
        tmp_str = "["+node
        if dag.get_parents(node)!=[]:
            tmp_str = tmp_str+"|"
            index = len
            for i in range(len(dag.get_parents(node))):
                if i == len(dag.get_parents(node))-1:
                    tmp_str+=dag.get_parents(node)[i]
                else:
                    tmp_str+=dag.get_parents(node)[i]+":"


        tmp_str+="]"
        rdag_str += tmp_str
    return rdag_str
    

In [3]:
ro.r("""
    data_tofactor<-function(data){

        datacols = names(data)
        
        for (i in 1:ncol(data)) {
        
            data[,datacols[i]] <- factor(data[,datacols[i]])
        }
    return(data)
    }
""")

#R结构
ro.r("""
        
        r_sturct_study<-function(self_r_data,bl,hos_id,scr){
        
            library(bnlearn)
            
            self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
            train_size = ceiling(length(self_r_data[,1])*0.7)
            
            train_data=self_r_data[1:train_size,]
            

            dag = hc(train_data,score = "k2",blacklist = bl)
            
            arcs = arc.strength(dag,train_data)
            
            return(arcs)
        }
""")

#R结构
ro.r("""
        
        r_sturct_study_wl<-function(self_r_data,bl,wl,hos_id,scr){
        
            library(bnlearn)
            
            self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
            train_size = ceiling(length(self_r_data[,1])*0.7)
            
            train_data=self_r_data[1:train_size,]
            

            dag = hc(train_data,score = "k2",whitelist=wl,blacklist = bl)
            
            arcs = arc.strength(dag,train_data)
            
            return(arcs)
        }
""")

#定义个一个用来预测的R语言的函数
ro.r("""
    predict_label<-function(r_dag,self_r_data,hos_id){
        library(bnlearn)
        
        model = model2network(r_dag)
        
        self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
        train_size = ceiling(length(self_r_data[,1])*0.7)
            
   
        
        train_data = self_r_data[1:train_size,nodes(model)]
        
        test_data = self_r_data[(train_size+1):nrow(self_r_data),nodes(model)]
    
        training = bn.fit(model,train_data)
        
        predicted = predict(training, node = "label", data = test_data)
        
        y_true = test_data[,"label"]
        
        out <- c(predicted ,y_true)
        
        return(out)
    }
""")
## 网络结构的评分
ro.r("""
    dag_score<-function(dag,self_r_data,score,hos_id){
    
    model = model2network(dag)
    self_r_data = self_r_data[self_r_data["hospitalid"]==hos_id,-1]
            
    train_size = ceiling(length(self_r_data[,1])*0.7)
            
    train_data=self_r_data[1:train_size,nodes(model)]
   
    scr = score(model, train_data, type =score)
    
    return(scr)
    }

""")

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x000002BB80B0C500> [RTYPES.CLOSXP]
R classes: ('function',)

In [4]:
class client():
    """
    只改动模型训练部分
    black和whitelist 的名单修改
    
    """
    def __init__(self,data,cid,scr):
        #black 用R语言的存储方式存储
        self.score = {}
        self.cid = cid
        self.data=data
        #初始化创建一个空的图
        self.model = DAG()
        self.scr = scr
        feature = data.columns
        
        self.edges_strength = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
        #创建记录评分的字典
        self.score = {
            "acc":[],
            "recall":[],
            "auc":[],
            "f1":[],
            "score":[]}
        self.dag_score=0
    def bl_update(self,bl):
        #在实例化客户端之后，根据自己数据的互信息值创建黑名单
        self.black_list = caculate_hos_bl(self.data,bl)
    def get_model(self):
        #获得本地客户端网络结构
        return self.model
    def fit(self):
        #=
        data,t_data = train_test_split(self.data,test_size=0.3,random_state=2)
        self_r_data = ro.pandas2ri.py2rpy(data)
        
        self_r_black_list = ro.conversion.py2rpy(self.black_list)
        
        
        arc_strength = ro.r['r_sturct_study'](DB,self_r_black_list,self.cid,self.scr)
        arc_strength = ro.pandas2ri.rpy2py_dataframe(arc_strength)
        
        self.arc_strength =  arc_strength.round(4)
        #创建新的模型，并添加结果
        self.model = DAG()
        print("客户端",self.cid,"开始fit")
        for i in range(len(arc_strength["from"])):
            
            #汇集频数
            self.model.add_edge(arc_strength["from"][i],arc_strength["to"][i])
          
            
#            #汇集强度转换为矩阵
            self.edges_strength.loc[[arc_strength["from"][i]],[arc_strength["to"][i]]]=arc_strength["strength"][i]
        self.edges_strength=self.edges_strength.round(4)
        
        rdag_str = py_to_rdag(self.model)
        #计算网路得分
        
        
        new_score = net_score(self.model,DB,self.cid,self.scr)
        
        self.score["acc"].append(new_score[0])
        self.score["recall"].append(new_score[1])
        self.score["auc"].append(new_score[2])
        self.score["f1"].append(new_score[3])
        self.score["score"].append(new_score[4])
        

        
        
        self.dag_score=new_score[4]
        return 1
    
    def update_model(self,wl):
        #代改动部分，每次也是创建一个新的贝叶斯网络实例进行白名单结构学习，并返回边模型，最好还是保持pgmpy的形式
        print(self.cid,"开始更新")
        new_dag = DAG()
        
        wl_r =ro.conversion.py2rpy(wl)
        #客户端将自身的黑名单转换为R数据模式传进函数里
        
        self_r_black_list = ro.conversion.py2rpy(self.black_list)
                                                 
        
        #调用R函数，并接收返回值
        
        arc_strength_r = ro.r['r_sturct_study_wl'](DB,self_r_black_list,wl_r,self.cid,self.scr)
        
        self.arc_strength=ro.pandas2ri.rpy2py_dataframe(arc_strength_r)
        
     
        #重新生成一个副本，比较那个性能较高
        for i in range(len(self.arc_strength["from"])):
            #转换自己的网络
            new_dag.add_edge(self.arc_strength["from"][i],self.arc_strength["to"][i])
            #生成新的强度列表
            self.edges_strength.loc[[self.arc_strength["from"][i]],[self.arc_strength["to"][i]]]=self.arc_strength["strength"][i]
        
        
        self.edges_strength=self.edges_strength.round(4)
        
        
        ##目前用AUC计算网络的分数
        
        new_score = net_score(new_dag,DB,self.cid,self.scr)
        
        old_score = net_score(self.model,DB,self.cid,self.scr)
        
        
        
        if new_score[4]>old_score[4]:
            self.model = new_dag
        else:
            new_score = old_score
            
      
        
        self.score["acc"].append(new_score[0])
        self.score["recall"].append(new_score[1])
        self.score["auc"].append(new_score[2])
        self.score["f1"].append(new_score[3])
        self.score["score"].append(new_score[4])
        
        self.dag_score=new_score[4]
        
    def client_score():
        ##获得当前本网络的性能
        r_dag = py_to_rdag(self.model)
        
        r_y = ro.r['predict_label'](r_dag,self_r_data,hos_id)
        split_size = len(r_y)*0.5
        y_pred = r_y[:split_size]
        y_true = r_y[split_size:]
        
        
        
class sever():
    def __init__(self):
        pass
        
    def Fd_caculate(self,edges_strength_matrix,round_num):
        #聚合方法
        #聚合的方式改为计算强度的阈值，并返回一个有无环图
        
        #变为list，并且设置为df格式
        arc_ls = pd.DataFrame(to_strength_list(edges_strength_matrix.astype(np.float16)),columns=["from","to","strength"])
        #from和to的df格式并按大小排列
        arc_ls = arc_ls.sort_values(by="strength")
        #1.按照10个选择，但是白名单的列表应该不断的成长，而不是固定的数值
        #在每次讲融合DAG加入网络之后，先检验是否有这条边，如果有则跳过
    
        i = 0
        index = 0

        fuseDAG = DAG()
        while i <round_num*5:
            arc = (arc_ls.loc[i+index,"from"],arc_ls.loc[i+index,"to"])
            ##防止图的有环结构
            if fuseDAG.has_edge(arc[1],arc[0]):
                i=i+1
                continue
            fuseDAG.add_edge(arc[0],arc[1])
            
         
            i+=1
        #检查边是否有环，并且破环
        
        print("检查环")
        
        for root in fuseDAG.nodes():
            node_list=fuseDAG.get_ancestral_graph(root).nodes()
            for nd in node_list:
                if fuseDAG.has_edge(root,nd):
                    print(root,nd)
                    fuseDAG.remove_edge(root,nd)
        return  fuseDAG

In [5]:
DB_1 = pd.read_csv("c:data/site_16topaucoutlinePmmImpData_dis.csv").drop(columns="Unnamed: 0",axis = 1)
# DB_1 = pd.read_csv("D:/实验室/数据/eicu数据处理/tenSiteDTDisc.csv").drop(columns=["Unnamed: 0","creatinine","patientunitstayid"],axis = 1)
DB_1=DB_1.sample(frac=1).reset_index(drop=True)
DB_1["label"].value_counts()
DB_1 = DB_1.astype(np.int16)
DB_1.fillna(1)
# 数据分割
# hos_id = [420,142,122,435,390,227,195,243,403,141]
hos_id = [420,142,122,435,390,227,144,140,396,141]##性能最好
# hos_id = [420,142,122,435,390,382,144,140,396,141]
# hos_id = [420,142,122,384,390,227,144,140,396,141]
# hos_id = [420,142,122,435,390,227,144,384,143,403]
# hos_id = [420,142,122,435,384,143,403,195,382,243]##性能最后后六个是外部验证
hos_id = np.sort(hos_id)
client_data = []
feature = DB_1.drop(columns = ["hospitalid"]).columns
global DB
DB = ro.r["data_tofactor"](DB_1)
for hosid in hos_id:
    client_data.append(DB_1[DB_1["hospitalid"]==hosid ].iloc[:,:].drop(columns = ["hospitalid"],axis = 1))
    
#设置黑名单
bl =[]
for i in feature:
    bl.append(["label",i])
    bl.append([i,"sex"])
    bl.append([i,"race"])
    bl.append([i,"bmi"])
    bl.append([i,"age"])
bl = pd.DataFrame(bl,columns=["from","to"])
# bl_r = ro.conversion.py2rpy(bl)
# globalenv['bl_r'] = bl_r
#计算样本量比重
D = 19388
D_W = []
for i in range(10):
    D_W.append(len(client_data[i])/D)
D_W = np.float16(D_W)

#将所有的离散状态记录下来，以字典的方式
state_names  ={
    "lable":[0,1],
    "age":[1,2,3,4,5,6,7,8],
    "sex":[1,2],
    "race":[1,2,3,4,5],
    "bmi":[1,2,3,4],
    "temperature":[1,2,3],
    "heartrate":[1,2,3],
    "respiration":[1,2,3],
    "SBP":[1,2,3,4,5],
    "DBP":[1,2,3,4,5],
    "paSystolic":[1,2,3],
    "paDiastolic":[1,2,3],
    "paMean":[1,2,3],
}
LAB_list = []
for i in range(1,61):
    LAB_list.append(("LAB"+str(i),[1,2,3]))
COM_list = []
for i in range(1,12):
    COM_list.append(("COM"+str(i),[0,1]))
PRO_list = []
for i in range(1,10):
    COM_list.append(("PRO"+str(i),[0,1]))
MED_list = []
for i in range(1,55):
    MED_list.append(("MED"+str(i),[0,1]))
state_names.update(LAB_list)
state_names.update(COM_list)
state_names.update(PRO_list)
state_names.update(MED_list)
BN_list = []
for i in range(1,11):
    BN_list.append("BN"+str(i))
    

In [6]:
all_score_init=[]
all_score_list=[]
fuse_dag_list = []
dag_struct_init_list = []
pandas2ri.activate()
scr_l=['fnml','bdla','mbde','bds','bic','loglik','bde', 'mbde', 'k2', 'qnml']
for scr in scr_l:
    if scr!="k2":
        continue
    fuseDAG = DAG()
    
    # 结构训练
    train_round = 10
    round_num = 1
    client_num = 10
    clients = []

    for i in range(10):
    # 创建客户端，并且传入数据和结构学习模型
        clients.append(client(client_data[i],hos_id[i],scr))
    # 初始化服务器
    sever_1 = sever()
    #分别创建一个0/1矩阵和强度矩阵
    edges_strength_matrix_list =[]
    #记录边的频数
    global_edges_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    #记录边的强度
    edges_strength_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)
    
    global_edges_strength_matrix = pd.DataFrame(np.zeros((len(feature),len(feature))),index = feature,columns=feature).astype(np.int16)



    #先本地客户端建立初始模型,并且计算每个网络的初始性能
    print("计算每个网络的初始性能")
    for i in range(client_num):
        clients[i].bl_update(bl)
        clients[i].fit()
        dag_struct_init_list.append(clients[i].model)
       
    #正式循环
    print("开始训练")
    while(round_num<=train_round):
        #重置矩阵,设置评分
        global_edges_matrix.iloc[:,:] = 0
        edges_strength_matrix.iloc[:,:] = 0
        #get到每个客户端边强度的总和
        arc_strength_list = []
        arc_weight_list = []
        dag_score_list= []
        #开始聚集客户端边的信息

        for i in range(client_num):
            #将客户端最好的模型边信息转换为矩阵形式
            global_edges_matrix+=to_matrix(clients[i].model.edges(),feature)
            dag_score_list.append(clients[i].dag_score)
            #将矩阵转换为列表形式，并strength按列合并数据
        #在这里计算边的强度
        #进行分数的权重
        dag_score_list=np.array(dag_score_list)
        min_score = min(dag_score_list)

        
        #汇集带权重的边的信息
        for i in range(client_num):
            #边的强度*数据量权重*网络分数/遍的个数 
    #         print(get_label_str(clients[i]))

            global_edges_strength_matrix+=clients[i].edges_strength
            ##融合网络权重的问题
            edges_strength_matrix+=clients[i].edges_strength*D_W[i]*dag_score_list[i]/len(clients[i].model.edges())


        #将矩阵转换为，一个from->to + str 的表


        global_edges_strength_matrix_list = to_strength_list(global_edges_strength_matrix)
     


        edges_strength_matrix_list.append(edges_strength_matrix)
    #     print("")
    #     for i in range(len(edges_strength_matrix_list)):
    #         print(edges_strength_matrix_list.iloc[i,:])

        fuseDAG = sever_1.Fd_caculate(edges_strength_matrix.round(4),round_num)
        
        

#         #输出一下混合的结果
#         for j in fuseDAG.edges:
#             print(j[0],"->",j[1],";")



        #得到fuseDAG之后，观察整个网络评分的变化

        print("第",round_num+1,"次迭代")
        #跟新客户端

        #设置白名单
        wl = []
        for edge in fuseDAG.edges():
            wl.append([edge[0],edge[1]])

        wl = pd.DataFrame(wl,columns=["from","to"])
        print("传递客户端信息")
        for i in range(client_num):

            clients[i]. update_model(wl)

        round_num+=1
        
 

    
    
    



计算每个网络的初始性能
客户端 122 开始fit
客户端 140 开始fit
客户端 141 开始fit
客户端 142 开始fit
客户端 144 开始fit
客户端 227 开始fit
客户端 390 开始fit
客户端 396 开始fit
客户端 420 开始fit
客户端 435 开始fit
开始训练
检查环
第 2 次迭代
传递客户端信息
122 开始更新
140 开始更新
141 开始更新
142 开始更新
144 开始更新
227 开始更新
390 开始更新
396 开始更新
420 开始更新
435 开始更新
检查环
第 3 次迭代
传递客户端信息
122 开始更新
140 开始更新
141 开始更新
142 开始更新
144 开始更新
227 开始更新
390 开始更新
396 开始更新
420 开始更新
435 开始更新
检查环
第 4 次迭代
传递客户端信息
122 开始更新
140 开始更新
141 开始更新
142 开始更新
144 开始更新
227 开始更新
390 开始更新
396 开始更新
420 开始更新
435 开始更新
检查环
第 5 次迭代
传递客户端信息
122 开始更新
140 开始更新
141 开始更新
142 开始更新
144 开始更新
227 开始更新
390 开始更新
396 开始更新
420 开始更新
435 开始更新
检查环
第 6 次迭代
传递客户端信息
122 开始更新
140 开始更新
141 开始更新
142 开始更新
144 开始更新
227 开始更新
390 开始更新
396 开始更新
420 开始更新
435 开始更新
检查环
第 7 次迭代
传递客户端信息
122 开始更新
140 开始更新
141 开始更新
142 开始更新
144 开始更新
227 开始更新
390 开始更新
396 开始更新
420 开始更新
435 开始更新
检查环
第 8 次迭代
传递客户端信息
122 开始更新
140 开始更新
141 开始更新
142 开始更新
144 开始更新
227 开始更新
390 开始更新
396 开始更新
420 开始更新
435 开始更新
检查环
第 9 次迭代
传递客户端信息
122 开始更新
140 开始更新
141 开始更新
142 开始更新
144 开始更新
227 开始更新

In [7]:
updated_list = []
init_list = []
for i in range(client_num):
    updated_list.append([
        clients[i].score["acc"][train_round-1],
        clients[i].score["recall"][train_round-1],
        clients[i].score["auc"][train_round-1],
        clients[i].score["f1"][train_round-1],
        clients[i].score["score"][train_round-1]])
    init_list.append([
        clients[i].score["acc"][0],
        clients[i].score["recall"][0],
        clients[i].score["auc"][0],
        clients[i].score["f1"][0],
        clients[i].score["score"][0]])
updated_score  = pd.DataFrame(updated_list,index=hos_id,columns=["acc",'recall','auc','f1','score']).T
init_score = pd.DataFrame(init_list,index=hos_id,columns=["acc",'recall','auc','f1','score']).T

In [8]:
#计算融合网络的性能
score_list=[]
for i in range(10):
    score_list.append(net_score(fuseDAG,DB,hos_id[i],scr))
fuse_score  = pd.DataFrame(score_list,index=hos_id,columns=["acc",'recall','auc','f1','score']).T

In [9]:
import plotly
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True) 
import plotly.graph_objs as go

In [10]:
def show_line(init,update,fuse,scoring):
    
    updated_scr = go.Scatter(
        x = [1,2,3,4,5,6,7,8,9,10],
        y =np.array(update.loc[scoring,:]),
        name = 'updated_client',
        mode = "lines+markers")
    
    fuse_scr = go.Scatter(
        x = [1,2,3,4,5,6,7,8,9,10],
        y =np.array(fuse.loc[scoring,:]),
        name = 'fuse_dag',
        mode = "lines+markers")

    init_scr = go.Scatter(
        x = [1,2,3,4,5,6,7,8,9,10],
        y =np.array(init.loc[scoring,:]),
        name = 'init_client',
        mode = "lines+markers")


    layout = go.Layout(
    xaxis=dict(
            tickvals = [1,2,3,4,5,6,7,8,9,10],
            ticktext=hos_id),
    title='模型迭代前后对比'+scoring,
        titlefont={
        'size':25,
        'color':'#9ed900'
        })
    data=[updated_scr,fuse_scr,init_scr]
    fig = go.Figure(data=data,layout=layout)
    iplot(fig)

In [11]:
for i in ["acc","recall","auc","f1","score"]:
    show_line(init_score,updated_score,fuse_score,i)